In [12]:
from boxoffice.db.frames import get_box_office_day_frame

print('hello world')

# bodf = get_box_office_day_frame()

# if bodf is None:
#     print('No box office data found')
#     exit()

# # group by movie id and sum the box office
# bodf = bodf.groupby('movie_id').sum()

# # sort by box office
# bodf = bodf.sort_values(by='box_office', ascending=False)

# print(bodf)

ModuleNotFoundError: No module named 'boxoffice'